In [204]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import statsmodels.api as sm

from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

from dateutil.relativedelta import relativedelta

pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

from IPython.display import HTML
from IPython.core.interactiveshell import InteractiveShell
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle
data = pd.read_csv("./data/kc_house_data.csv")

In [205]:
# Load Data From Pickel
with open('regression_Initial_Model_sk.pickle', 'rb') as file:
    data_Model1_sk = pickle.load(file)
    
with open('scaler_1.pickle', 'rb') as file:
    scaler_1 = pickle.load(file)    

with open('data_Target_Homes_Not_Scaled_WithDummies.pickle', 'rb') as file:
    data_Target_Homes_Not_Scaled_WithDummies = pickle.load(file)    

#need to create new pickle file of variables that we start with when we scale vs. the end
# with open('model1_Continuous_variables.pickle', 'rb') as file:
#     model1_Continuous_variables = pickle.load(file)    

In [206]:
data_Target_Homes_Not_Scaled_WithDummies.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,view_2,view_3,view_4,condition_3,condition_4,condition_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,55,981,2014,12.00,13.311329,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925,55,980,2015,7.50,13.403776,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,9212900260,5/27/2014,468000.0,2,1.00,1160,6000,1.0,860,300.0,1942,0.0,98115,47.6900,-122.292,1330,6000,78,981,2014,2.00,13.056224,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,9297300055,1/24/2015,650000.0,4,3.00,2950,5000,2.0,1980,970.0,1979,0.0,98126,47.5714,-122.375,2140,4000,41,981,2015,12.00,13.384728,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [207]:
colsToScale = ['bedrooms', 'bathrooms','sqft_living','sqft_lot','sqft_basement','sqft_living15', 'sqft_lot15', 'floors',
               'sqft_above','AgeOfHome','ttl_rooms']
#append sc_ to name of cal
colsToScale_with_sc = ["sc_" + e for e in colsToScale]
# colsToScale_with_sc = list(map(lambda s: s.strip(), colsToScale_with_sc))

In [208]:
colsToScale_with_sc

['sc_bedrooms',
 'sc_bathrooms',
 'sc_sqft_living',
 'sc_sqft_lot',
 'sc_sqft_basement',
 'sc_sqft_living15',
 'sc_sqft_lot15',
 'sc_floors',
 'sc_sqft_above',
 'sc_AgeOfHome',
 'sc_ttl_rooms']

In [209]:
# df_Scaled_Data_test = pd.DataFrame(data=Scaled_Data_1, columns=[colsToScale_with_sc])

In [210]:
# df_Scaled_Data_test.columns

In [211]:
data_Target_Homes_colsToScale = data_Target_Homes_Not_Scaled_WithDummies[colsToScale]

In [212]:
Scaled_Data_1 = scaler_1.transform(data_Target_Homes_colsToScale)
df_Scaled_Data_1 = pd.DataFrame(data=Scaled_Data_1, columns=['sc_bedrooms', 'sc_bathrooms','sc_sqft_living',
                                                             'sc_sqft_lot','sc_sqft_basement','sc_sqft_living15', 
                                                             'sc_sqft_lot15', 'sc_floors','sc_sqft_above','sc_AgeOfHome',
                                                             'sc_ttl_rooms'])

In [213]:
df_Scaled_Data_1.columns

Index(['sc_bedrooms', 'sc_bathrooms', 'sc_sqft_living', 'sc_sqft_lot',
       'sc_sqft_basement', 'sc_sqft_living15', 'sc_sqft_lot15', 'sc_floors',
       'sc_sqft_above', 'sc_AgeOfHome', 'sc_ttl_rooms'],
      dtype='object')

In [214]:
data_Target_Homes_Not_Scaled_WithDummies.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,view_2,view_3,view_4,condition_3,condition_4,condition_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,55,981,2014,12.00,13.311329,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925,55,980,2015,7.50,13.403776,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,9212900260,5/27/2014,468000.0,2,1.00,1160,6000,1.0,860,300.0,1942,0.0,98115,47.6900,-122.292,1330,6000,78,981,2014,2.00,13.056224,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,9297300055,1/24/2015,650000.0,4,3.00,2950,5000,2.0,1980,970.0,1979,0.0,98126,47.5714,-122.375,2140,4000,41,981,2015,12.00,13.384728,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [215]:
#Combined scaled values with original values
data_Target_Homes_Not_Scaled_WithDummies = data_Target_Homes_Not_Scaled_WithDummies.reset_index(drop=True)
# df_Scaled_Data_1 = df_Scaled_Data_1.reset_index(drop=True)
data_Target_Homes_Scaled_WithDummies = pd.concat([data_Target_Homes_Not_Scaled_WithDummies, df_Scaled_Data_1], axis=1)


In [216]:
data_Target_Homes_Scaled_WithDummies.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,...,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,sc_bedrooms,sc_bathrooms,sc_sqft_living,sc_sqft_lot,sc_sqft_basement,sc_sqft_living15,sc_sqft_lot15,sc_floors,sc_sqft_above,sc_AgeOfHome,sc_ttl_rooms
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.250,0.3125,0.464373,0.540333,0.184332,0.305970,0.235309,0.5,0.528125,0.556522,0.145570
1,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,55,981,2014,12.00,13.311329,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.375,0.5000,0.314496,0.166667,0.419355,0.223881,0.142054,0.0,0.178125,0.434783,0.278481
2,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925,55,980,2015,7.50,13.403776,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.250,0.3750,0.707617,0.966000,0.783410,0.435323,0.280752,0.0,0.431250,0.434783,0.164557
3,9212900260,5/27/2014,468000.0,2,1.00,1160,6000,1.0,860,300.0,1942,0.0,98115,47.6900,-122.292,1330,6000,78,981,2014,2.00,13.056224,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.125,0.0000,0.117936,0.333333,0.138249,0.216418,0.177391,0.0,0.118750,0.634783,0.025316
4,9297300055,1/24/2015,650000.0,4,3.00,2950,5000,2.0,1980,970.0,1979,0.0,98126,47.5714,-122.375,2140,4000,41,981,2015,12.00,13.384728,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.375,0.5000,0.557740,0.166667,0.447005,0.417910,0.106718,0.5,0.468750,0.313043,0.278481


In [217]:
#Get and insert the average value for each feature for easy retravial when adding values to demonstrate regression
def resetAverages():
    df_Averages = pd.DataFrame()
    df_Averages['a_bedrooms'] = [data_Target_Homes_Not_Scaled_WithDummies['bedrooms'].mean()]
    df_Averages['a_bathrooms'] = [data_Target_Homes_Not_Scaled_WithDummies['bathrooms'].mean()]
    df_Averages['a_sqft_living'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_living'].mean()]
    df_Averages['a_sqft_lot'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_lot'].mean()]
    df_Averages['a_sqft_basement'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_basement'].mean()]
    df_Averages['a_sqft_living15'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_living15'].mean()]
    df_Averages['a_sqft_lot15'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_lot15'].mean()]
    df_Averages['a_floors'] = [data_Target_Homes_Not_Scaled_WithDummies['floors'].mean()]
    df_Averages['a_sqft_above'] = [data_Target_Homes_Not_Scaled_WithDummies['sqft_above'].mean()]
    df_Averages['a_AgeOfHome'] = [data_Target_Homes_Not_Scaled_WithDummies['AgeOfHome'].mean()]
    df_Averages['a_ttl_rooms'] = [data_Target_Homes_Not_Scaled_WithDummies['ttl_rooms'].mean()]
    df_Averages['a_waterfront_1'] = [0]
    df_Averages['a_view_1'] = [0]
    df_Averages['a_view_2'] = [0]
    df_Averages['a_view_3'] = [0]
    df_Averages['a_view_4'] = [0]
    df_Averages['a_condition_4'] = [0]
    df_Averages['a_condition_5'] = [0]
    df_Averages['a_grade_6'] = [0]
    df_Averages['a_grade_8'] = [0]
    df_Averages['a_grade_9'] = [0]
    df_Averages['a_grade_10'] = [0]
    df_Averages['a_grade_11'] = [0]
    df_Averages['a_ZipFirst3_981'] =[0]
    df_Averages['a_sls_mnth_2'] = [0]
    df_Averages['a_sls_mnth_3'] = [0]
    df_Averages['a_sls_mnth_4'] = [0]
    df_Averages['a_sls_mnth_5'] = [1]
    df_Averages['a_sls_mnth_6'] = [0]
    df_Averages['a_sls_mnth_7'] = [0]
    df_Averages['a_sls_mnth_8'] = [0]
    df_Averages['a_sls_mnth_9'] = [0]
    df_Averages['a_sls_mnth_10'] = [0]
    df_Averages['a_sls_mnth_11'] = [0]
    df_Averages['a_sls_mnth_12'] = [0]
    df_Averages['a_If_renovated_1'] = [0]
    df_Averages['a_sc_bedrooms'] = data_Target_Homes_Scaled_WithDummies['sc_bedrooms'].mean()
    df_Averages['a_sc_bathrooms'] = data_Target_Homes_Scaled_WithDummies['sc_bathrooms'].mean()
    df_Averages['a_sc_sqft_living'] =data_Target_Homes_Scaled_WithDummies['sc_sqft_living'].mean()
    df_Averages['a_sc_sqft_lot'] = data_Target_Homes_Scaled_WithDummies['sc_sqft_lot'].mean()
    df_Averages['a_sc_sqft_basement'] = data_Target_Homes_Scaled_WithDummies['sc_sqft_basement'].mean()
    df_Averages['a_sc_sqft_living15'] = data_Target_Homes_Scaled_WithDummies['sc_sqft_living15'].mean()
    df_Averages['a_sc_sqft_lot15'] = data_Target_Homes_Scaled_WithDummies['sc_sqft_lot15'].mean()
    df_Averages['a_sc_floors'] = data_Target_Homes_Scaled_WithDummies['sc_floors'].mean()
    df_Averages['a_sc_sqft_above'] = data_Target_Homes_Scaled_WithDummies['sc_sqft_above'].mean()
    df_Averages['a_sc_AgeOfHome'] = data_Target_Homes_Scaled_WithDummies['sc_AgeOfHome'].mean()
    df_Averages['a_sc_ttl_rooms'] = data_Target_Homes_Scaled_WithDummies['sc_ttl_rooms'].mean()
    return df_Averages


In [218]:
test = resetAverages()

In [219]:
test

,a_bedrooms,a_bathrooms,a_sqft_living,a_sqft_lot,a_sqft_basement,a_sqft_living15,a_sqft_lot15,a_floors,a_sqft_above,a_AgeOfHome,a_ttl_rooms,a_waterfront_1,a_view_1,a_view_2,a_view_3,a_view_4,a_condition_4,a_condition_5,a_grade_6,a_grade_8,a_grade_9,a_grade_10,a_grade_11,a_ZipFirst3_981,a_sls_mnth_2,a_sls_mnth_3,a_sls_mnth_4,a_sls_mnth_5,a_sls_mnth_6,a_sls_mnth_7,a_sls_mnth_8,a_sls_mnth_9,a_sls_mnth_10,a_sls_mnth_11,a_sls_mnth_12,a_If_renovated_1,a_sc_bedrooms,a_sc_bathrooms,a_sc_sqft_living,a_sc_sqft_lot,a_sc_sqft_basement,a_sc_sqft_living15,a_sc_sqft_lot15,a_sc_floors,a_sc_sqft_above,a_sc_AgeOfHome,a_sc_ttl_rooms
0,3.550367,2.079419,2095.072495,6928.795819,715.459951,1849.946239,7025.376052,1.166848,1379.612544,60.961444,7.672957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.318796,0.269855,0.347684,0.488133,0.329705,0.345758,0.213625,0.083424,0.281129,0.486621,0.168936


In [220]:
X_train = data_Target_Homes_Scaled_WithDummies[['id','waterfront_1', 'view_1', 'view_2', 'view_3', 'view_4',
       'condition_4', 'condition_5', 'grade_6', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'ZipFirst3_981', 'sls_mnth_2', 'sls_mnth_3',
       'sls_mnth_4', 'sls_mnth_5', 'sls_mnth_6', 'sls_mnth_7', 'sls_mnth_8',
       'sls_mnth_9', 'sls_mnth_10', 'sls_mnth_11', 'sls_mnth_12',
       'If_renovated_1', 'sc_sqft_lot', 'sc_sqft_basement', 'sc_sqft_above']]


In [221]:
X_train_with_ID = X_train.loc[X_train['id']==2114700040]

In [222]:
X_train_with_ID.head()
X_train_No_ID = X_train_with_ID.drop(columns=["id"], axis=1)
X_train_No_ID.head()

,waterfront_1,view_1,view_2,view_3,view_4,condition_4,condition_5,grade_6,grade_8,grade_9,grade_10,grade_11,ZipFirst3_981,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,sc_sqft_lot,sc_sqft_basement,sc_sqft_above
2460,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.021833,0.115207,0.3125


In [223]:
#Feed value into model and get a predictions
# 'waterfront_1', 'view_1', 'view_2', 'view_3', 'view_4',
#        'condition_4', 'condition_5', 'grade_6', 'grade_8', 'grade_9',
#        'grade_10', 'grade_11', 'ZipFirst3_981', 'sls_mnth_2', 'sls_mnth_3',
#        'sls_mnth_4', 'sls_mnth_5', 'sls_mnth_6', 'sls_mnth_7', 'sls_mnth_8',
#        'sls_mnth_9', 'sls_mnth_10', 'sls_mnth_11', 'sls_mnth_12',
#        'If_renovated_1', 'sc_sqft_lot', 'sc_sqft_basement', 'sc_sqft_above'
            


y_hat = data_Model1_sk.predict(X_train_No_ID)
print(y_hat)
print(np.exp(y_hat))

[13.20039944]
[540580.82628751]


In [224]:
data_Target_Homes_Scaled_WithDummies.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,...,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,sc_bedrooms,sc_bathrooms,sc_sqft_living,sc_sqft_lot,sc_sqft_basement,sc_sqft_living15,sc_sqft_lot15,sc_floors,sc_sqft_above,sc_AgeOfHome,sc_ttl_rooms
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.250,0.3125,0.464373,0.540333,0.184332,0.305970,0.235309,0.5,0.528125,0.556522,0.145570
1,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,55,981,2014,12.00,13.311329,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.375,0.5000,0.314496,0.166667,0.419355,0.223881,0.142054,0.0,0.178125,0.434783,0.278481
2,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925,55,980,2015,7.50,13.403776,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.250,0.3750,0.707617,0.966000,0.783410,0.435323,0.280752,0.0,0.431250,0.434783,0.164557
3,9212900260,5/27/2014,468000.0,2,1.00,1160,6000,1.0,860,300.0,1942,0.0,98115,47.6900,-122.292,1330,6000,78,981,2014,2.00,13.056224,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.125,0.0000,0.117936,0.333333,0.138249,0.216418,0.177391,0.0,0.118750,0.634783,0.025316
4,9297300055,1/24/2015,650000.0,4,3.00,2950,5000,2.0,1980,970.0,1979,0.0,98126,47.5714,-122.375,2140,4000,41,981,2015,12.00,13.384728,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.375,0.5000,0.557740,0.166667,0.447005,0.417910,0.106718,0.5,0.468750,0.313043,0.278481


In [225]:
# X_train_No_ID_Average = data_Target_Homes_Scaled_WithDummies [['a_sc_waterfront_1', 'a_sc_view_1', 'a_sc_view_2', 
#                                                                'a_sc_view_3', 'a_sc_view_4', 'a_sc_condition_4',
#                                                                'a_sc_condition_5', 'a_sc_grade_6', 'a_sc_grade_8', 
#                                                                'a_sc_grade_9', 'a_sc_grade_10', 'a_sc_grade_11',
#                                                                'a_sc_ZipFirst3_981', 'a_sc_sls_mnth_2', 
#                                                                'a_sc_sls_mnth_3', 'a_sc_sls_mnth_4', 
#                                                                'a_sc_sls_mnth_5','a_sc_sls_mnth_6', 'a_sc_sls_mnth_7', 
#                                                                'a_sc_sls_mnth_8', 'a_sc_sls_mnth_9', 'a_sc_sls_mnth_10',
#                                                                'a_sc_sls_mnth_11', 'a_sc_sls_mnth_12', 'a_sc_If_renovated_1', 'a_sc_sqft_lot',
#                                                                'a_sc_sqft_basement', 'a_sc_sqft_above']]

# y_hat_average = data_Model1_sk.predict(X_train_No_ID_Average.head(1))
# print(y_hat_average)
# print(np.exp(y_hat_average))

In [226]:
# data_Target_Homes_Scaled_WithDummies.head(

In [227]:
def Calculate_New_Unscaled_Value_For_TargetCol(TargetColumn, deltaInValue,id=0):
    
    if id == 0:
        Target_UnscaledValue_ForTargetCol = Get_Average_UnscaledValue_ForTargetCol(TargetColumn)
        print (f' Target Column {TargetColumn} - ')
        print (f' Here {Target_UnscaledValue_ForTargetCol} - ')
    else:
        Target_Home = data_Target_Homes_Not_Scaled_WithDummies.loc[data_Target_Homes_Not_Scaled_WithDummies["id"]==id]
#         X_train_with_ID = X_train.loc[X_train['id']==2114700040]
        Target_UnscaledValue_ForTargetCol = Target_Home[TargetColumn]
    
    UpdatedValue_ForTargetCol = Target_UnscaledValue_ForTargetCol + deltaInValue
    
#     targetCol = "a_" + ColumnToChange
#     targetValue_NotScaled = df_Averages[targetCol]
#     Scaled_Data_1 = scaler_1.transform(data_Target_Homes_colsToScale)
    
    return UpdatedValue_ForTargetCol

In [228]:
# Target_Home = data_Target_Homes_Not_Scaled_WithDummies.loc[data_Target_Homes_Not_Scaled_WithDummies["id"]==6414100192]
# test = Target_Home['sqft_above']

In [229]:
# test

In [230]:
df_Averages

,a_bedrooms,a_bathrooms,a_sqft_living,a_sqft_lot,a_sqft_basement,a_sqft_living15,a_sqft_lot15,a_floors,a_sqft_above,a_AgeOfHome,a_ttl_rooms,a_waterfront_1,a_view_1,a_view_2,a_view_3,a_view_4,a_condition_4,a_condition_5,a_grade_6,a_grade_8,a_grade_9,a_grade_10,a_grade_11,a_ZipFirst3_981,a_sls_mnth_2,a_sls_mnth_3,a_sls_mnth_4,a_sls_mnth_5,a_sls_mnth_6,a_sls_mnth_7,a_sls_mnth_8,a_sls_mnth_9,a_sls_mnth_10,a_sls_mnth_11,a_sls_mnth_12,a_If_renovated_1,a_sc_bedrooms,a_sc_bathrooms,a_sc_sqft_living,a_sc_sqft_lot,a_sc_sqft_basement,a_sc_sqft_living15,a_sc_sqft_lot15,a_sc_floors,a_sc_sqft_above,a_sc_AgeOfHome,a_sc_ttl_rooms
0,3.550367,2.079419,2095.072495,6928.795819,715.459951,1849.946239,7025.376052,1.166848,2270,60.961444,7.672957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.318796,0.269855,0.347684,0.488133,0.329705,0.345758,0.213625,0.083424,0.281129,0.486621,0.168936


In [231]:
def Get_Average_UnscaledValue_ForTargetCol(TargetColumn):
    targetCol = "a_" + TargetColumn
    for (columnName, columnData) in df_Averages.iteritems(): 
        if columnName == targetCol:
            Average_ofTargetValue = columnData.values
#             print(f'Average of target value = {Average_ofTargetValue}') 
            return Average_ofTargetValue

In [232]:
def Get_UnscaledValue_ForTargetCol(TargetColumn):
#     targetCol = "a_" + TargetColumn
    for (columnName, columnData) in data_Target_Homes_Not_Scaled_WithDummies.iteritems(): 
        if columnName == targetCol:
            TargetValue = columnData.values
#             print(Average_ofTargetValue) 
            return TargetValue
#         print('Colunm Name : ', columnName) 
#         print('Column Contents : ', columnData.values)

In [233]:
data_Target_Homes_Not_Scaled_WithDummies.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,view_2,view_3,view_4,condition_3,condition_4,condition_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,55,981,2014,12.00,13.311329,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925,55,980,2015,7.50,13.403776,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,9212900260,5/27/2014,468000.0,2,1.00,1160,6000,1.0,860,300.0,1942,0.0,98115,47.6900,-122.292,1330,6000,78,981,2014,2.00,13.056224,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,9297300055,1/24/2015,650000.0,4,3.00,2950,5000,2.0,1980,970.0,1979,0.0,98126,47.5714,-122.375,2140,4000,41,981,2015,12.00,13.384728,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [247]:
testing = data_Target_Homes_Scaled_WithDummies.loc[data_Target_Homes_Scaled_WithDummies.id==6414100192]
testing

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,...,grade_10,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,sc_bedrooms,sc_bathrooms,sc_sqft_living,sc_sqft_lot,sc_sqft_basement,sc_sqft_living15,sc_sqft_lot15,sc_floors,sc_sqft_above,sc_AgeOfHome,sc_ttl_rooms
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.721,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.25,0.3125,0.464373,0.540333,0.184332,0.30597,0.235309,0.5,0.528125,0.556522,0.14557


In [253]:
#Get New Price based on Column, Average or Actual

df_Averages = resetAverages()
TargetColumn = "sqft_above"
targetCol = "a_" + TargetColumn
deltaInValue = 100
# parcelID = 0
parcelID = 6414100192

NewValue = Calculate_New_Unscaled_Value_For_TargetCol(TargetColumn,100,parcelID)

if parcelID ==0:
    #we are calculating for an avearge
    df_Averages[targetCol] = NewValue
    df_target_data_to_scale = df_Averages
    colsToScale = ['a_bedrooms', 'a_bathrooms','a_sqft_living','a_sqft_lot','a_sqft_basement','a_sqft_living15', 'a_sqft_lot15',
               'a_floors','a_sqft_above','a_AgeOfHome','a_ttl_rooms']
    colsToFeed_In_to_Predict = ['a_waterfront_1', 'a_view_1', 'a_view_2', 'a_view_3', 'a_view_4', 'a_condition_4',
       'a_condition_5', 'a_grade_6', 'a_grade_8', 'a_grade_9', 'a_grade_10', 'a_grade_11',
       'a_ZipFirst3_981', 'a_sls_mnth_2', 'a_sls_mnth_3', 'a_sls_mnth_4', 'a_sls_mnth_5',
       'a_sls_mnth_6', 'a_sls_mnth_7', 'a_sls_mnth_8', 'a_sls_mnth_9', 'a_sls_mnth_10',
       'a_sls_mnth_11', 'a_sls_mnth_12', 'a_If_renovated_1', 'n_sc_sqft_lot',
       'n_sc_sqft_basement', 'n_sc_sqft_above']


else:
    df_target_data_to_scale = data_Target_Homes_Not_Scaled_WithDummies.loc[data_Target_Homes_Not_Scaled_WithDummies['id']==parcelID]
    df_target_data_to_scale[TargetColumn] = NewValue
    
    colsToScale = ['bedrooms', 'bathrooms','sqft_living','sqft_lot','sqft_basement','sqft_living15', 'sqft_lot15',
               'floors','sqft_above','AgeOfHome','ttl_rooms']
    colsToFeed_In_to_Predict = ['waterfront_1', 'view_1', 'view_2', 'view_3', 'view_4', 'condition_4',
       'condition_5', 'grade_6', 'grade_8', 'grade_9', 'grade_10', 'grade_11',
       'ZipFirst3_981', 'sls_mnth_2', 'sls_mnth_3', 'sls_mnth_4', 'sls_mnth_5',
       'sls_mnth_6', 'sls_mnth_7', 'sls_mnth_8', 'sls_mnth_9', 'sls_mnth_10',
       'sls_mnth_11', 'sls_mnth_12', 'If_renovated_1', 'n_sc_sqft_lot',
       'n_sc_sqft_basement', 'n_sc_sqft_above']


Scaled_Data_1 = scaler_1.transform(df_target_data_to_scale[colsToScale])

df_Scaled_Data_1 = pd.DataFrame(data=Scaled_Data_1, columns=['n_sc_bedrooms', 'n_sc_bathrooms','n_sc_sqft_living',
                                                             'n_sc_sqft_lot','n_sc_sqft_basement','n_sc_sqft_living15', 
                                                             'n_sc_sqft_lot15', 'n_sc_floors','n_sc_sqft_above','n_sc_AgeOfHome',
                                                             'n_sc_ttl_rooms'])

#Combined scaled values with original values
df_target_data_to_scale = df_target_data_to_scale.reset_index(drop=True)
# df_Scaled_Data_1 = df_Scaled_Data_1.reset_index(drop=True)

data_TargetHomes_With_Updated_TargetColumn = pd.concat([df_target_data_to_scale, df_Scaled_Data_1], axis=1)



X_train_No_ID_Average = data_TargetHomes_With_Updated_TargetColumn [colsToFeed_In_to_Predict]

y_hat_average = data_Model1_sk.predict(X_train_No_ID_Average)
print(y_hat_average)
print(np.exp(y_hat_average))


[13.20759799]
[544486.26256953]


<ipython-input-253-19f5eca779db>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_data_to_scale[TargetColumn] = NewValue


In [252]:
test = df_target_data_to_scale[colsToScale]
test

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_basement,sqft_living15,sqft_lot15,floors,sqft_above,AgeOfHome,ttl_rooms
0,3,2.25,2570,7242,400.0,1690,7639,2.0,2170,69,6.75


In [250]:
NewValue

0    2270
Name: sqft_above, dtype: int64

In [249]:
X_train_No_ID_Average

,waterfront_1,view_1,view_2,view_3,view_4,condition_4,condition_5,grade_6,grade_8,grade_9,grade_10,grade_11,ZipFirst3_981,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,n_sc_sqft_lot,n_sc_sqft_basement,n_sc_sqft_above
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.540333,0.184332,0.528125


In [244]:
data_TargetHomes_With_Updated_TargetColumn.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,AgeOfHome,ZipFirst3_int,salesyear,ttl_rooms,log_price,waterfront_1,ZipFirst3_981,view_1,...,grade_11,sls_mnth_2,sls_mnth_3,sls_mnth_4,sls_mnth_5,sls_mnth_6,sls_mnth_7,sls_mnth_8,sls_mnth_9,sls_mnth_10,sls_mnth_11,sls_mnth_12,If_renovated_1,a_sqft_above,n_sc_bedrooms,n_sc_bathrooms,n_sc_sqft_living,n_sc_sqft_lot,n_sc_sqft_basement,n_sc_sqft_living15,n_sc_sqft_lot15,n_sc_floors,n_sc_sqft_above,n_sc_AgeOfHome,n_sc_ttl_rooms
0,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,2170,400.0,1951,1991.0,98125,47.721,-122.319,1690,7639,69,981,2014,6.75,13.195614,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,2270,0.25,0.3125,0.464373,0.540333,0.184332,0.30597,0.235309,0.5,0.528125,0.556522,0.14557


In [240]:
NewValue

0    2270
Name: sqft_above, dtype: int64

In [ ]:
# get column, add qty, rescale, repredict
# df_AverageValues = data_Target_Homes_Scaled_WithDummies.head(1)

test = FindingChangeInAverageHome("sqft_above",100)
test
# X_train_No_ID_Average = data_Target_Homes_Scaled_WithDummies [['a_sc_waterfront_1', 'a_sc_view_1', 'a_sc_view_2', 'a_sc_view_3', 'a_sc_view_4', 'a_sc_condition_4',
#        'a_sc_condition_5', 'a_sc_grade_6', 'a_sc_grade_8', 'a_sc_grade_9', 'a_sc_grade_10', 'a_sc_grade_11',
#        'a_sc_ZipFirst3_981', 'a_sc_sls_mnth_2', 'a_sc_sls_mnth_3', 'a_sc_sls_mnth_4', 'a_sc_sls_mnth_5',
#        'a_sc_sls_mnth_6', 'a_sc_sls_mnth_7', 'a_sc_sls_mnth_8', 'a_sc_sls_mnth_9', 'a_sc_sls_mnth_10',
#        'a_sc_sls_mnth_11', 'a_sc_sls_mnth_12', 'a_sc_If_renovated_1', 'a_sc_sqft_lot',
#        'a_sc_sqft_basement', 'a_sc_sqft_above']]
